In [0]:
from __future__ import print_function
import time
import seaborn as sns
import os
import random
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_mldata
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.utils.data
import torchvision.utils as vutils
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms
from torchvision import datasets
from torchvision.utils import save_image

from google.colab import drive
drive.mount('/content/gdrive')


class DRCN(nn.Module):
    def __init__(self, n_class):
        super(DRCN, self).__init__()

        # convolutional encoder

        self.enc_feat = nn.Sequential()
        self.enc_feat.add_module('conv1', nn.Conv2d(in_channels=1, out_channels=100, kernel_size=5,
                                                    padding=2))
        self.enc_feat.add_module('relu1', nn.ReLU(True))
        self.enc_feat.add_module('pool1', nn.MaxPool2d(kernel_size=2, stride=2))

        self.enc_feat.add_module('conv2', nn.Conv2d(in_channels=100, out_channels=150, kernel_size=5,
                                                    padding=2))
        self.enc_feat.add_module('relu2', nn.ReLU(True))
        self.enc_feat.add_module('pool2', nn.MaxPool2d(kernel_size=2, stride=2))

        self.enc_feat.add_module('conv3', nn.Conv2d(in_channels=150, out_channels=200, kernel_size=3,
                                                    padding=1))
        self.enc_feat.add_module('relu3', nn.ReLU(True))

        self.enc_dense = nn.Sequential()
        self.enc_dense.add_module('fc4', nn.Linear(in_features=200 * 8 * 8, out_features=1024))
        self.enc_dense.add_module('relu4', nn.ReLU(True))
        self.enc_dense.add_module('drop4', nn.Dropout2d())

        self.enc_dense.add_module('fc5', nn.Linear(in_features=1024, out_features=1024))
        self.enc_dense.add_module('relu5', nn.ReLU(True))

        # label predict layer
        self.pred = nn.Sequential()
        self.pred.add_module('dropout6', nn.Dropout2d())
        self.pred.add_module('predict6', nn.Linear(in_features=1024, out_features=n_class))

        # convolutional decoder

        self.rec_dense = nn.Sequential()
        self.rec_dense.add_module('fc5_', nn.Linear(in_features=1024, out_features=1024))
        self.rec_dense.add_module('relu5_', nn.ReLU(True))

        self.rec_dense.add_module('fc4_', nn.Linear(in_features=1024, out_features=200 * 8 * 8))
        self.rec_dense.add_module('relu4_', nn.ReLU(True))

        self.rec_feat = nn.Sequential()

        self.rec_feat.add_module('conv3_', nn.Conv2d(in_channels=200, out_channels=150,
                                                     kernel_size=3, padding=1))
        self.rec_feat.add_module('relu3_', nn.ReLU(True))
        self.rec_feat.add_module('pool3_', nn.Upsample(scale_factor=2))

        self.rec_feat.add_module('conv2_', nn.Conv2d(in_channels=150, out_channels=100,
                                                     kernel_size=5, padding=2))
        self.rec_feat.add_module('relu2_', nn.ReLU(True))
        self.rec_feat.add_module('pool2_', nn.Upsample(scale_factor=2))

        self.rec_feat.add_module('conv1_', nn.Conv2d(in_channels=100, out_channels=1,
                                                     kernel_size=5, padding=2))

    def forward(self, input_data):
        feat = self.enc_feat(input_data)
        feat = feat.view(-1, 200 * 8 * 8)
        feat_code = self.enc_dense(feat)

        pred_label = self.pred(feat_code)

        feat_encode = self.rec_dense(feat_code)
        feat_encode = feat_encode.view(-1, 200, 8, 8)
        img_rec = self.rec_feat(feat_encode)

        return pred_label, img_rec, feat_code

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Test on the MNIST test dataset
def test(epoch):

    model_root = '/content/gdrive/My Drive'
    image_root = os.path.join('/content/gdrive/My Drive/dataset', 'mnist')

    cuda = True
    cudnn.benchmark = True
    batch_size = 64
    image_size = 32

    # load data
    img_transform = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor()
    ])

    dataset = datasets.MNIST(
        root=image_root,
        train=False,
        download=True,
        transform=img_transform
    )

    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8
    )

    # test
    my_net = torch.load(os.path.join(
        model_root, 'svhn_mnist_model_epoch_' + str(epoch) + '.pth')
    )

    my_net = my_net.eval()
    if cuda:
        my_net = my_net.cuda()

    len_dataloader = len(data_loader)
    data_iter = iter(data_loader)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_dataloader:

        data = data_iter.next()
        img, label = data

        batch_size = len(label)

        input_img = torch.FloatTensor(batch_size, 1, image_size, image_size)
        class_label = torch.LongTensor(batch_size)

        if cuda:
            img = img.cuda()
            label = label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()

        input_img.resize_as_(img).copy_(img)
        class_label.resize_as_(label).copy_(label)
        inputv_img = Variable(input_img)
        classv_label = Variable(class_label)

        pred_label, _, _ = my_net(input_data=inputv_img)
        pred = pred_label.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(classv_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size

        i += 1

    accu = n_correct * 1.0 / n_total

    print ('epoch: %d, mnist test accuracy: %f' %(epoch, accu))

In [0]:
# Test on the SVHN test dataset
def rec_image(epoch):

    model_root = '/content/gdrive/My Drive'
    image_root = os.path.join('/content/gdrive/My Drive/dataset', 'svhn')

    cuda = True
    cudnn.benchmark = True
    batch_size = 64
    image_size = 32

    # load data
    img_transfrom = transforms.Compose([
        transforms.Resize(32),
        transforms.Grayscale(),
        transforms.ToTensor()
    ])

    dataset = datasets.SVHN(
        root=image_root,
        split='test',
        download=True,
        transform=img_transfrom
    )

    data_loader = torch.utils.data.DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=8
    )

    # test
    my_net = torch.load(os.path.join(
        model_root, 'svhn_mnist_model_epoch_' + str(epoch) + '.pth')
    )
    i = 0
    n_total = 0
    n_correct = 0

    my_net = my_net.eval()
    if cuda:
        my_net = my_net.cuda()

    data_iter = iter(data_loader)
    data = data_iter.next()
    img, label = data
    batch_size = len(img)

    input_img = torch.FloatTensor(batch_size, 1, image_size, image_size)
    class_label = torch.LongTensor(batch_size)

    if cuda:
        img = img.cuda()
        label = label.cuda()
        input_img = input_img.cuda()
        class_label = class_label.cuda()

    input_img.resize_as_(img).copy_(img)
    inputv_img = Variable(input_img)
    class_label.resize_as_(label).copy_(label)
    classv_label = Variable(class_label)

    pred_label, rec_img, _ = my_net(input_data=inputv_img)
    pred = pred_label.data.max(1, keepdim=True)[1]
    n_correct += pred.eq(classv_label.data.view_as(pred)).cpu().sum()
    n_total += batch_size

    accu = n_correct * 1.0 / n_total
    print ('epoch: %d, svhn test accuracy: %f' %(epoch, accu))

    # vutils.save_image(input_img, '/content/gdrive/My Drive/svhn_real_epoch_' + str(epoch) + '.png', nrow=8)
    # vutils.save_image(rec_img.data, '/content/gdrive/My Drive/svhn_rec_' + str(epoch) + '.png', nrow=8)

In [0]:
source_dataset_name = 'SVHN'
target_dataset_name = 'mnist'
source_dataset = os.path.join('/content/gdrive/My Drive/dataset', 'svhn')
target_dataset = os.path.join('/content/gdrive/My Drive/dataset', 'mnist')
model_root = '/content/gdrive/My Drive'   # directory to save trained models
cuda = True
cudnn.benchmark = True
lr = 1e-4
batch_size = 64
image_size = 32
n_epoch = 20
weight_decay = 5e-6
m_lambda = 0.7

# weights initialisation with Xavier Uniform initializer
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform(m.weight.data, gain=1)
        nn.init.constant(m.bias.data, 0.1)

manual_seed = random.randint(1, 10000)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# load data
img_transform_svhn = transforms.Compose([
    transforms.Resize(32),
    transforms.Grayscale(),
    transforms.RandomRotation(20),
    transforms.ToTensor()
])

img_transform_mnist = transforms.Compose([
    transforms.Resize(32),
    transforms.RandomRotation(20),
    transforms.ToTensor()
])

dataset_source = datasets.SVHN(
    root=source_dataset,
    split='train',
    download=True,
    transform=img_transform_svhn,
)

datasetloader_source = torch.utils.data.DataLoader(
    dataset=dataset_source,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8
)

dataset_target = datasets.MNIST(
    root=target_dataset,
    train=True,
    download=True,
    transform=img_transform_mnist,
)

datasetloader_target = torch.utils.data.DataLoader(
    dataset=dataset_target,
    batch_size=batch_size,
    shuffle=True,
    num_workers=8
)

# load model
my_net = DRCN(n_class=10)
my_net.apply(weights_init)

# setup Adam optimizer
optimizer_classify = optim.Adam([{'params': my_net.enc_feat.parameters()},
                                    {'params': my_net.enc_dense.parameters()},
                                    {'params': my_net.pred.parameters()}], lr=lr, weight_decay=weight_decay)

optimizer_rec = optim.Adam([{'params': my_net.enc_feat.parameters()},
                               {'params': my_net.enc_dense.parameters()},
                               {'params': my_net.rec_dense.parameters()},
                               {'params': my_net.rec_feat.parameters()}], lr=lr, weight_decay=weight_decay)

loss_class = nn.CrossEntropyLoss()
loss_rec = nn.MSELoss()

if cuda:
    my_net = my_net.cuda()
    loss_class = loss_class.cuda()
    loss_rec = loss_rec.cuda()

for p in my_net.parameters():
    p.requires_grad = True

len_source = len(datasetloader_source)
len_target = len(datasetloader_target)

Using downloaded and verified file: /content/gdrive/My Drive/dataset/svhn/train_32x32.mat


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.


In [0]:
# training
for epoch in range(n_epoch):

    # train a convolutional autoencoder for target data reconstruction
    dataset_target_iter = iter(datasetloader_target)

    i = 0
    n_total = 0
    n_correct = 0

    while i < len_target:
        my_net.zero_grad()

        data_target = dataset_target_iter.next()
        t_img, _ = data_target

        batch_size = len(t_img)

        input_img = torch.FloatTensor(batch_size, 1, image_size, image_size)

        if cuda:
            t_img = t_img.cuda()
            input_img = input_img.cuda()

        input_img.resize_as_(t_img).copy_(t_img)
        inputv_img = Variable(input_img)

        _, rec_img, latent_img = my_net(input_data=inputv_img)
        # save_image(rec_img.data, './recovery_image/mnist_rec' + str(epoch) + '.png', nrow=8)

        rec_img = rec_img.view(-1, 1 * image_size * image_size)
        inputv_img_img = inputv_img.contiguous().view(-1, 1 * image_size * image_size)
        shape = list(rec_img.size())
        inputv_img = inputv_img.resize(shape[0], 1024)
        err_rec = (1 - m_lambda) * loss_rec(rec_img, inputv_img)
        err_rec.backward()
        optimizer_rec.step()

        i += 1

    # print ('epoch: %d, err_rec %f' \
    #       % (epoch, err_rec.cpu().data.numpy()))

    # train a convolutional network for label prediction based on the source data

    dataset_source_iter = iter(datasetloader_source)

    i = 0
    latent_img_list = []

    while i < len_source:
        my_net.zero_grad()

        data_source = dataset_source_iter.next()
        s_img, s_label = data_source
        s_label = s_label.long().squeeze()

        batch_size = len(s_label)

        input_img = torch.FloatTensor(batch_size, 1, image_size, image_size)
        class_label = torch.LongTensor(batch_size)
        if cuda:
            s_img = s_img.cuda()
            s_label = s_label.cuda()
            input_img = input_img.cuda()
            class_label = class_label.cuda()


        input_img.resize_as_(s_img).copy_(s_img)
        class_label.resize_as_(s_label).copy_(s_label)
        inputv_img = Variable(input_img)
        classv_label = Variable(class_label)

        pred_label, _, latent_img = my_net(input_data=inputv_img)
        err_class = m_lambda * loss_class(pred_label, classv_label)
        err_class.backward()
        optimizer_classify.step()
        pred = pred_label.data.max(1, keepdim=True)[1]
        n_correct += pred.eq(classv_label.data.view_as(pred)).cpu().sum()
        n_total += batch_size
        latent_img_list.append(latent_img)

        i += 1
    
    accu = n_correct * 1.0 / n_total
    print ('epoch: %d, svhn train accuracy: %f' %(epoch, accu))

    # latent_img_list = torch.cat(latent_img_list).cpu().detach().numpy() #data.cpu().numpy()
    # print(latent_img_list.shape)
    # X_embedded = TSNE(n_components=2).fit_transform(latent_img_list)

    torch.save(my_net, '{0}/svhn_mnist_model_epoch_{1}.pth'.format(model_root, epoch))

    rec_image(epoch)
    test(epoch)

print ('done')

/usr/local/lib/python3.6/dist-packages/torch/tensor.py:330: UserWarning: non-inplace resize is deprecated
  warnings.warn("non-inplace resize is deprecated")


epoch: 0, svhn train accuracy: 0.534857


/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type DRCN. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Sequential. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type Conv2d. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type ReLU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-pa

Using downloaded and verified file: /content/gdrive/My Drive/dataset/svhn/test_32x32.mat
epoch: 0, svhn test accuracy: 0.843750
epoch: 0, mnist test accuracy: 0.674700
epoch: 1, svhn train accuracy: 0.757907
Using downloaded and verified file: /content/gdrive/My Drive/dataset/svhn/test_32x32.mat
epoch: 1, svhn test accuracy: 0.859375
epoch: 1, mnist test accuracy: 0.712500
epoch: 2, svhn train accuracy: 0.801248
Using downloaded and verified file: /content/gdrive/My Drive/dataset/svhn/test_32x32.mat
epoch: 2, svhn test accuracy: 0.859375
epoch: 2, mnist test accuracy: 0.727000
epoch: 3, svhn train accuracy: 0.818297
Using downloaded and verified file: /content/gdrive/My Drive/dataset/svhn/test_32x32.mat
epoch: 3, svhn test accuracy: 0.812500
epoch: 3, mnist test accuracy: 0.704600
epoch: 4, svhn train accuracy: 0.834023
Using downloaded and verified file: /content/gdrive/My Drive/dataset/svhn/test_32x32.mat
epoch: 4, svhn test accuracy: 0.875000
epoch: 4, mnist test accuracy: 0.719100
